### **0. Подготовка обучающей и тестовой выборки**

In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
%matplotlib inline 
sns.set(style="ticks")

In [29]:
# Будем использовать только обучающую выборку
data = pd.read_csv('data/train.csv', sep=",")

In [30]:
X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]  # Пример признаков
y = data['Survived']  # Целевая переменная (1 - выжил, 0 - не выжил)
# Разделение на train и test (сначала split, потом обработка, чтобы избежать data leakage)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
data = data.dropna(subset=['Embarked'])
data = data.drop('Cabin', axis=1)

In [32]:
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Заполняем пропуски медианой
    ('scaler', StandardScaler())  # Масштабируем
])

In [34]:
categorical_features = ['Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Заполняем пропуски модой
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Кодируем категории
])

In [37]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [38]:
# Применяем преобразования
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [42]:
X_train_processed

array([[-1.91971935, -0.47416141,  1.99885349, ...,  1.        ,
         0.        ,  0.        ],
       [-0.0772525 , -0.47416141, -0.47932706, ...,  0.        ,
         0.        ,  1.        ],
       [-2.15002771,  0.34868694,  0.75976322, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.92075038,  1.17153529, -0.47932706, ...,  0.        ,
         0.        ,  1.        ],
       [-1.15202483,  0.34868694,  1.99885349, ...,  1.        ,
         0.        ,  0.        ],
       [-0.61463866, -0.47416141,  0.75976322, ...,  1.        ,
         0.        ,  0.        ]])

### **1. Обучение KNN с произвольным K**

Этап обучения: модель просто запоминает все обучающие данные (никакиих вычислений)

In [45]:
from sklearn.neighbors import KNeighborsClassifier

# Обучаем модель с K=5 (произвольный выбор)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_processed, y_train)

KNeighborsClassifier()

Этап предсказания: для нового объекта вычисляется евклидово расстояние до всех точек в обучающей выборке, выбирается **K** соседей и выбирается класс который чаще встречается среди этих соседей

In [46]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Предсказание и оценка
y_pred = knn.predict(X_test_processed)
print("Accuracy (K=5):", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy (K=5): 0.8134328358208955
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       157
           1       0.81      0.71      0.76       111

    accuracy                           0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268

Confusion Matrix:
 [[139  18]
 [ 32  79]]


Формулки чтоб не забыть:
1) Accuracy = (TP + TN) / (TP + TN + FP + FN)
2) Precision = TP / (TP + FP) сколько предсказаний этого класса верны
3) Recall = TP / (TP + FN) сколько представителей класса мы отыскали
4) F1-score = 2 * (precision * recall) / (precision + recall) среднее гармоническое

### **2. Подбор параметра K**

#### **2.1 GridSearchCV (полный перебор)**

In [53]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
param_grid = {'n_neighbors': range(1, 30)}
grid_search = GridSearchCV(
    KNeighborsClassifier(),
    param_grid,
    cv=StratifiedKFold(n_splits=5),
    scoring='accuracy'
)
grid_search.fit(X_train_processed, y_train)

print("Лучший K (GridSearch):", grid_search.best_params_)
print("Лучшая accuracy (GridSearch):", grid_search.best_score_)

Лучший K (GridSearch): {'n_neighbors': 24}
Лучшая accuracy (GridSearch): 0.8250064516129033


#### **2.2 RandomizedSearchCV (случайный поиск)**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
random_search = RandomizedSearchCV(
    KNeighborsClassifier(),
    {'n_neighbors': range(1, 50)},
    n_iter=15,
    cv=KFold(n_splits=5),  #5-Fold кросс валидация, для каждого К данные делятся на 5 частей, обучение на 4ех, проверка на 5ой, процесс повторяется 5 раз (каждая часть по очереди - валидационная)
    scoring='accuracy',
    random_state=42
)
random_search.fit(X_train_processed, y_train)

print("Лучший K (RandomizedSearch):", random_search.best_params_)
print("Лучшая accuracy (RandomizedSearch):", random_search.best_score_)

Лучший K (RandomizedSearch): {'n_neighbors': 14}
Лучшая accuracy (RandomizedSearch): 0.8137935483870968


### **3. Сравнение исходной и оптимальной моделей**

In [57]:
# Оптимальная модель (например, из GridSearchCV)
best_knn = grid_search.best_estimator_
y_pred_best = best_knn.predict(X_test_processed)

print("\nИсходная модель (K=5):")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

print("\nОптимальная модель (K={}):".format(grid_search.best_params_['n_neighbors']))
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))


Исходная модель (K=5):
Accuracy: 0.8134328358208955
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       157
           1       0.81      0.71      0.76       111

    accuracy                           0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268


Оптимальная модель (K=24):
Accuracy: 0.8059701492537313
              precision    recall  f1-score   support

           0       0.79      0.90      0.85       157
           1       0.83      0.67      0.74       111

    accuracy                           0.81       268
   macro avg       0.81      0.79      0.79       268
weighted avg       0.81      0.81      0.80       268

